In [3]:
import pandas as pd
import os
import glob
import timeit
import numpy as np

In [4]:
# Load cancer generation matrix
big_matrix = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_intermediate/complete_cancer_candidates_order_r_complete.tsv.gz'
mx = pd.read_csv(big_matrix, sep = '\t')
mx.head()

,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,cancerCohortfilter >=2.0,cancerCohortfilter >=3.0,cancerCohortfilter >=5.0,cancerCohortfilter >=10.0,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated
0,RKSTQMPCT,1218.0,689.0,249.0,89.0,26.0,4.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
1,RKSTQMPCT,1279.0,274.0,70.0,28.0,5.0,1.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
2,RKSTQMPCT,7350.0,4622.0,2653.0,1745.0,1126.0,734.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
3,RKSTQMPCT,2025.0,479.0,96.0,35.0,8.0,1.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
4,RKSTQMPCT,8641.0,6957.0,4789.0,3211.0,1347.0,191.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN


In [104]:
mx.head()

,coord,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,junctionAnnotated,readFrameAnnotated,...,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate1,junction_coordinate2,STAR_GTEx_jx
0,92379857:92379859:92611313:92611338:None:None,RKSTQMPCT,1218.0,689.0,249.0,89.0,26.0,4.0,True,False,...,20.982616,8.422932,17.304198,16.069005,19.585264,NaN,+,92379859:92611313,None,True
1,92379857:92379859:92611313:92611338:None:None,RKSTQMPCT,1279.0,274.0,70.0,28.0,5.0,1.0,True,False,...,20.982616,8.422932,17.304198,16.069005,19.585264,NaN,+,92379859:92611313,None,True
2,92379857:92379859:92611313:92611338:None:None,RKSTQMPCT,7350.0,4622.0,2653.0,1745.0,1126.0,734.0,True,False,...,20.982616,8.422932,17.304198,16.069005,19.585264,NaN,+,92379859:92611313,None,True
3,92379857:92379859:92611313:92611338:None:None,RKSTQMPCT,2025.0,479.0,96.0,35.0,8.0,1.0,True,False,...,20.982616,8.422932,17.304198,16.069005,19.585264,NaN,+,92379859:92611313,None,True
4,92379857:92379859:92611313:92611338:None:None,RKSTQMPCT,8641.0,6957.0,4789.0,3211.0,1347.0,191.0,True,False,...,20.982616,8.422932,17.304198,16.069005,19.585264,NaN,+,92379859:92611313,None,True


In [5]:
# Extract junction coordinates, sort them, split first and second junction
coord_mx = mx['coord'].str.split(':', expand=True) #7 min

coord_mx[1] = coord_mx[1].astype('int')
coord_mx[2] = coord_mx[2].astype('int')

coord_mx['strand'] = None
coord_mx.loc[coord_mx[1] < coord_mx[2], 'strand'] = '+'
coord_mx.loc[coord_mx[1] > coord_mx[2], 'strand'] = '-'

coord_mx['junction_coordinate1'] = None
coord_mx['junction_coordinate2'] = None


coord_mx = coord_mx.astype(str) # 7 min

coord_mx['+first'] = coord_mx[1] + ':' + coord_mx[2]
coord_mx['+secon'] = coord_mx[3] + ':' + coord_mx[4]
coord_mx['-first'] = coord_mx[3] + ':' + coord_mx[0]
coord_mx['-secon'] = coord_mx[5] + ':' + coord_mx[2]

coord_mx.loc[(coord_mx['strand'] == '+'), 'junction_coordinate1'] = coord_mx['+first'] 
coord_mx.loc[(coord_mx['strand'] == '-'), 'junction_coordinate1'] = coord_mx['-first'] 
coord_mx.loc[(coord_mx['strand'] == '+') & (coord_mx[4] != 'None') , 'junction_coordinate2'] = coord_mx['+secon']
coord_mx.loc[(coord_mx['strand'] == '-') & (coord_mx[4] != 'None') , 'junction_coordinate2'] = coord_mx['-secon']

In [13]:
coord_mx.head()

,0,1,2,3,4,5,strand,junction_coordinate1,junction_coordinate2,+first,+secon,-first,-secon
0,92379857,92379859,92611313,92611338,None,None,+,92379859:92611313,None,92379859:92611313,92611338:None,92611338:92379857,None:92611313
1,92379857,92379859,92611313,92611338,None,None,+,92379859:92611313,None,92379859:92611313,92611338:None,92611338:92379857,None:92611313
2,92379857,92379859,92611313,92611338,None,None,+,92379859:92611313,None,92379859:92611313,92611338:None,92611338:92379857,None:92611313
3,92379857,92379859,92611313,92611338,None,None,+,92379859:92611313,None,92379859:92611313,92611338:None,92611338:92379857,None:92611313
4,92379857,92379859,92611313,92611338,None,None,+,92379859:92611313,None,92379859:92611313,92611338:None,92611338:92379857,None:92611313


In [19]:
# Add split junction information to generation table
mx = pd.concat([mx, coord_mx[['strand', 'junction_coordinate1', 'junction_coordinate2']]], axis = 1)

In [20]:
# Load STAR junctions
star_jx = '/cluster/work/grlab/projects/GTEx/rna_gencode32_realign/results/junctions_spladder.all_coords.sorted.uniq.tsv.gz'
star_jx = pd.read_csv(star_jx, sep = '\t')
star_jx.head()

,chr,strand,junction_start,junction_end
0,chr10,+,100000101,100361248
1,chr10,-,100000120,100233082
2,chr10,-,100000237,100000286
3,chr10,+,100000266,100043158
4,chr10,-,100000336,100373716


In [ ]:
# Add STAR junction column 10 min
star_jx['junction_coordinate'] = star_jx['junction_start'].astype(str) + ':' + star_jx['junction_end'].astype(str)

# print(star_jx[['junction_start', 'junction_end']].drop_duplicates().shape)

# print(star_jx[['chr', 'junction_start', 'junction_end']].drop_duplicates().shape)

# print(star_jx.drop_duplicates().shape)

In [ ]:
print(mx.shape)

In [ ]:
# Kmers from 1 junction
one_jx = mx[(~ mx['junction_coordinate1'].isna()) & (mx['junction_coordinate2'].isna())]

print(one_jx.shape)

In [ ]:
# Kmers from 2 junctions
two_jx = mx[(~ mx['junction_coordinate1'].isna()) & (~ mx['junction_coordinate2'].isna())]
print(two_jx.shape)

In [ ]:
# NOT IN STAR 1 junction
isstar_one = set(one_jx['junction_coordinate1']).intersection(set(star_jx['junction_coordinate']))
print(len(isstar_one_jx))
is_star_one_jx = set(one_jx.set_index('junction_coordinate1').loc[isstar_one, 'coord'])

In [ ]:
# Not in STAR 2 junctions
is_star_two1 = set(two_jx['junction_coordinate1']).intersection(set(star_jx['junction_coordinate']))
is_star_two2 = set(two_jx['junction_coordinate2']).intersection(set(star_jx['junction_coordinate']))


is_star_two_jx = set(two_jx.set_index('junction_coordinate1').loc[is_star_two1, 'coord']).intersection(\
set(two_jx.set_index('junction_coordinate2').loc[is_star_two2, 'coord']))

In [ ]:
print(len(is_star_one_jx))
print(len(is_star_two_jx))

In [ ]:
# FLAG GTEX junctions
mx['STAR_GTEx_jx'] = False
mx = mx.set_index('coord')
mx.loc[list(is_star_one_jx.union(is_star_two_jx)), 'STAR_GTEx_jx'] = True
mx = mx.reset_index()

# Quality check

In [97]:
mx.tail()

,coord,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,junctionAnnotated,readFrameAnnotated,...,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate1,junction_coordinate2,STAR_GTEx_jx
46132790,121185317:121185343:121181347:121181348:None:None,GGETRSPEN,9017.0,8371.0,7325.0,6256.0,4444.0,1973.0,True,True,...,0.839305,0.842293,0.0,0.0,0.0,1.0,None,None,None,False
46132791,121185317:121185343:121181347:121181348:None:None,GGETRSPEN,7252.0,6130.0,4821.0,3703.0,2144.0,620.0,True,True,...,0.839305,0.842293,0.0,0.0,0.0,1.0,None,None,None,False
46132792,121185317:121185343:121181347:121181348:None:None,GGETRSPEN,8207.0,7083.0,5422.0,4020.0,2197.0,593.0,True,True,...,0.839305,0.842293,0.0,0.0,0.0,1.0,None,None,None,False
46132793,121185317:121185343:121181347:121181348:None:None,GGETRSPEN,7757.0,6386.0,4834.0,3713.0,2244.0,729.0,True,True,...,0.839305,0.842293,0.0,0.0,0.0,1.0,None,None,None,False
46132794,121185317:121185343:121181347:121181348:None:None,GGETRSPEN,7757.0,6256.0,4725.0,3632.0,2182.0,703.0,True,True,...,0.839305,0.842293,0.0,0.0,0.0,1.0,None,None,None,False


In [101]:
mx.loc[mx['coord'] ==  '1001544:1001563:1001877:1001881:1001981:1001985']

,coord,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,junctionAnnotated,readFrameAnnotated,...,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate1,junction_coordinate2,STAR_GTEx_jx
116471,1001544:1001563:1001877:1001881:1001981:1001985,VSMTMQGRV,904.0,887.0,545.0,363.0,102.0,10.0,True,False,...,0.839305,0.0,0.0,0.0,0.0,NaN,+,1001563:1001877,1001881:1001981,True


In [103]:
star_jx.loc[star_jx['junction_coordinate'] ==  '1001881:1001981']

,chr,strand,junction_start,junction_end,junction_coordinate
58862277,chr4,+,1001881,1001981,1001881:1001981
